# [0] 이론

## [0-1] 선형회귀 vs 이진 로지스틱 회귀 vs 다중 로지스틱 회귀 (다중 클래스 분류(클래스 중 1개))

### [0-1-1] 선형회귀  

![](images/선형회귀.png)

### [0-1-2] 이진 로지스틱 회귀  

![](images/이진정리.png)![](images/이진.png) 

## [0-1-3] 다중 로지스틱 회귀

- 다중 로지스틱 회귀 = multinomial = softmax 로지스틱 회귀

- 다중 로지스틱 회귀  
![](images/다중.png) ![](images/다중가설.png) ![](images/다중손실.png)
    - 해당 클래스가 답이면 (y=1), pi=1이면 맞는 답!
     - softmax
        - 각 클래스로 분류될 확률의 합이 1이 되도록 하는 함수 (0.4,0.7,0.2->0.3,0.5,0.2)
        ![](images/솦맥.png)

# [1] 학습 파이프라인

pipeline : 학습 모델을 위해 필요한 변환(transform)들을 하나의 파이프라인으로 모을 수 있는 사이킷런 객체
- fit과 predict 메소드를 적용할 수 있는 scaler, regression과 같은 객체들을 모을 수 있음
- Pipeline 함수를 통해서 생성
- steps 옵션에 해당 이름과 객체를 튜플로 작성해서 생성

- 생성 후에 fit과 predict를 통해 여러 step들을 한번에 적용할 수 있음


In [7]:
import numpy as np
from sklearn.datasets import load_breast_cancer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split

X, y = load_breast_cancer(return_X_y=True, as_frame=True)
X_train, X_test, y_train, y_test = train_test_split(X, y,test_size=0.33,random_state=1234)
y_train

470    1
484    1
509    0
271    1
384    1
      ..
279    1
372    0
204    1
53     0
294    1
Name: target, Length: 381, dtype: int64

In [4]:
scaler = StandardScaler()
clf = LogisticRegression(random_state=1234, solver='sag')
#pipe = Pipeline(steps=[ scaler, clf])
pipe = Pipeline(steps=[("scaler", scaler), ("clf", clf)])

# 스케일링과 학습 한꺼번에!
pipe.fit(X_train, y_train)

/opt/homebrew/lib/python3.11/site-packages/sklearn/utils/validation.py:767: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if not hasattr(array, "sparse") and array.dtypes.apply(is_sparse).any():
/opt/homebrew/lib/python3.11/site-packages/sklearn/utils/validation.py:605: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype):
/opt/homebrew/lib/python3.11/site-packages/sklearn/utils/validation.py:614: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype) or not is_extension_array_dtype(pd_dtype):
/opt/homebrew/lib/python3.11/site-packages/sklearn/utils/validation.py:767: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  i

Pipeline(steps=[('scaler', StandardScaler()),
                ('clf', LogisticRegression(random_state=1234, solver='sag'))])

In [8]:
y_pred = pipe.predict(X_test)

print(f'정확도:{(y_pred == y_test).mean() * 100: .2f}%')

정확도: 94.68%


/opt/homebrew/lib/python3.11/site-packages/sklearn/utils/validation.py:767: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if not hasattr(array, "sparse") and array.dtypes.apply(is_sparse).any():
/opt/homebrew/lib/python3.11/site-packages/sklearn/utils/validation.py:605: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype):
/opt/homebrew/lib/python3.11/site-packages/sklearn/utils/validation.py:614: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype) or not is_extension_array_dtype(pd_dtype):


# [2] 다중 로지스틱 예제-파이프라인

In [15]:
import numpy as np
from sklearn.datasets import load_iris
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import precision_score

X, y = load_iris(return_X_y=True, as_frame=True)
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.33,random_state=1234)
y_test

91     1
63     1
103    2
6      0
59     1
29     0
27     0
35     0
99     1
122    2
82     1
42     0
117    2
61     1
43     0
79     1
100    2
25     0
108    2
94     1
74     1
54     1
51     1
73     1
109    2
48     0
123    2
92     1
102    2
11     0
98     1
126    2
40     0
133    2
77     1
24     0
8      0
20     0
22     0
93     1
36     0
68     1
32     0
141    2
125    2
9      0
114    2
145    2
135    2
140    2
Name: target, dtype: int64

In [16]:
scaler = MinMaxScaler()
clf = LogisticRegression(random_state=1234, solver='saga')
pipe = Pipeline(steps=[("scaler", scaler), ("clf", clf)])

# 한꺼번에 [스케일링+학습 -> 예측]
y_pred = pipe.fit(X_train, y_train).predict(X_test)
print(X_test.shape)
# 정밀도 (average 매개변수를 None으로 설정했기 때문에 각 클래스의 정밀도를 개별적으로 계산)
pre = precision_score(y_test, y_pred, average = None)
print(f'정밀도:{pre[1]*100: .2f}%')

(50, 4)
정밀도: 94.12%


/opt/homebrew/lib/python3.11/site-packages/sklearn/utils/validation.py:767: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if not hasattr(array, "sparse") and array.dtypes.apply(is_sparse).any():
/opt/homebrew/lib/python3.11/site-packages/sklearn/utils/validation.py:605: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype):
/opt/homebrew/lib/python3.11/site-packages/sklearn/utils/validation.py:614: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype) or not is_extension_array_dtype(pd_dtype):
/opt/homebrew/lib/python3.11/site-packages/sklearn/utils/validation.py:767: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  i

# [3] 다중 로지스틱 예제-LogisticRegression

LogisticRegression 객체 옵션
- penalty: 모델의 복잡도를 조절하는 방법(정규화 유형 지정)
    - 'l1', 'l2'(default)
- C: 페널티 정도
- solver: 다양한 해결 방법을 고를 있음
- max_iter: 반복 횟수
- multi_class : ‘auto’, ‘ovr’, ‘multinomial’ 
    - ‘ovr’: 각 클래스에 대해 하나의 이진 로지스틱 회귀 적용
    - ‘multinomial’: softmax 로지스틱회귀
    - ‘auto’: 데이터에 따라 자동으로 적용

LogisticRegression 객체 속성
- classes: 클래스 리스트
- n_features_in_: 피처개수
- coef_: (클래스개수, 피처개수) 만큼의 계수를 반환
- intercept_: (클래스개수,) 만큼의 절편을 반환

In [23]:
import numpy as np
import pandas as pd

############ 1. 데이터 만들기

n = 300
np.random.seed(1234)
X1 = np.random.normal(0, 1, size=n)
X2 = np.random.normal(0, 1, size=n)
X3 = np.random.normal(0, 1, size=n)
X4 = np.random.normal(0, 1, size=n)
X5 = np.random.normal(0, 1, size=n)

# x 데이터 만들기
X = pd.DataFrame({'X1': X1, 'X2': X2, 'X3': X3, 'X4': X4, 'X5': X5})
X.shape

(300, 5)

In [21]:
ys = []
intercept = -2
beta = np.array([[-2, -2, 3, 4, 5]]).reshape(-1, 1)

# y 데이터 만들기
np.random.seed(1111)
for i in range(n):
    xb = np.exp((intercept + (X.iloc[i].values.reshape(1, -1) @ beta)[0][0]) + 
                np.random.normal(0, 7.5))
    pi = xb / (1 + xb)  
    if pi >= 0.5: y = 1
    else: y = 0
    ys.append(y)
y = pd.Series(ys)

print(y)

0      0
1      0
2      0
3      0
4      1
      ..
295    0
296    0
297    0
298    0
299    0
Length: 300, dtype: int64


In [33]:
from sklearn.linear_model import LogisticRegression

############ 2. logisticregression으로 학습 및 예측

clf = LogisticRegression(random_state=1234)
clf = clf.fit(X, y)

print(f'절편: {clf.intercept_[0]}')
print(f'계수: {clf.coef_[0]}')
# 서윤추가 print
print(f'클래스 리스트: {clf.classes_}')
print(f'피처 개수: {clf.n_features_in_}')

y_pred = clf.predict(X)

print(f'정확도:{(y == y_pred).mean() * 100: .2f}%')

절편: -0.578686940895635
계수: [-0.52203159 -0.51494024  0.71392631  1.05416056  1.06007824]
클래스 리스트: [0 1]
피처 개수: 5
정확도: 78.00%


/opt/homebrew/lib/python3.11/site-packages/sklearn/utils/validation.py:767: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if not hasattr(array, "sparse") and array.dtypes.apply(is_sparse).any():
/opt/homebrew/lib/python3.11/site-packages/sklearn/utils/validation.py:605: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype):
/opt/homebrew/lib/python3.11/site-packages/sklearn/utils/validation.py:614: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype) or not is_extension_array_dtype(pd_dtype):
/opt/homebrew/lib/python3.11/site-packages/sklearn/utils/validation.py:605: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  i